In [1]:
import sklearn
print('Hi')
print(sklearn.__version__)

Hi
0.24.2


# Modelling

In [4]:
import pandas as pd
import sklearn as sk
import numpy as np
import pickle

In [5]:
known = 6
noise = 0
train = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_train_{known}_0.csv")
test = pd.read_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_test_{known}_{noise}.csv")

In [5]:
test.head() 

,input_0_time,input_0_x,input_0_y,input_1_time,input_1_x,input_1_y,input_2_time,input_2_x,input_2_y,output_x,output_y
0,30.0,0.0,0.0,18.0,-6.1,24.4,10.0,-11.2,41.6,-16.1,62.8
1,22.0,0.0,0.0,12.0,-4.4,21.8,4.0,-4.4,21.8,-10.8,48.1
2,30.0,0.0,0.0,20.0,-7.3,19.1,10.0,-15.1,38.9,-22.5,57.9
3,30.0,0.0,0.0,20.0,-5.9,17.9,10.0,-11.7,34.8,-18.0,52.1
4,30.0,0.0,0.0,22.0,-5.9,14.6,12.0,-10.6,33.6,-17.5,57.0


In [7]:
train_y = train.loc[:,["output_x","output_y"]]
test_y = test.loc[:,["output_x","output_y"]]
train_x = train.drop(columns=["output_x","output_y"])
test_x = test.drop(columns=["output_x","output_y"])

In [7]:
train_x.head()
#train_y.head(100)
#print(type("input_2_x"))

,input_0_time,input_0_x,input_0_y,input_1_time,input_1_x,input_1_y,input_2_time,input_2_x,input_2_y
0,22.0,0.0,0.0,21.0,0.0,0.0,11.0,-0.1,0.3
1,20.0,0.0,0.0,10.0,0.4,0.1,1.0,0.6,0.3
2,30.0,0.0,0.0,19.0,0.0,0.5,11.0,-0.2,0.4
3,20.0,0.0,0.0,19.0,0.0,0.0,9.0,-0.4,0.0
4,30.0,0.0,0.0,21.0,-0.6,0.3,11.0,-0.6,0.4


In [8]:
df_res = test_y.copy()

df_res["output_x"] = known
df_res["output_y"] = noise

#df_res.head()

## Baseline Testing

In [9]:
import numpy as np
import pandas as pd

# dummy_prediction = test_x.loc[:,["input_2_x", "input_2_y"]]
dummy_prediction = test_x.loc[:,[f"input_{known-1}_x", f"input_{known-1}_y"]]


#df_test = test.copy()

df_res[f"bl_{known}_{noise}"] = np.sqrt((test_y["output_x"] - dummy_prediction[f"input_{known-1}_x"])**2 + (test_y["output_y"] - dummy_prediction[f"input_{known-1}_y"])**2)

df_res.head()



#df_test['sqrt'] = pd.Series((np.sqrt(("input_2_x"- "output_x")**2 + ("input_2_y"- "output_y"))**2)), index=df_test.index)
#df_test['sqrt'] = pd.Series(0, index=df_test.index)
#df_test['sqrt'] = np.sqrt((df_test["input_2_x"] - df_test["output_x"])**2 + (df_test["input_2_y"] - df_test["output_y"])**2)
#df_test['sqrt'] = np.sqrt((df_test["input_2_x"] + df_test["output_x"]) + (df_test["input_2_y"] + df_test["output_y"]))
#print(f"Anzahl Schiffe: {df_test['sqrt'].nunique()}")
#baseline = dummy_prediction.groupby("sqrt").size()
print(df_res[f"bl_{known}_{noise}"].describe())

count    320568.000000
mean         30.464561
std          25.380031
min           0.000000
25%           5.423099
50%          30.647023
75%          50.480095
max         205.212695
Name: bl_3_0, dtype: float64


## Linear Regression

Funktionierende Modelle:
https://scikit-learn.org/1.5/modules/multiclass.html#multioutput-regression

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

linearpipe = Pipeline([("scaler", StandardScaler()), ("regressor", LinearRegression())])
linearpipe.fit(train_x, train_y)

#np.sqrt(test_y, linearpipe.predict(test_x))


#df_test = test.copy()
#df = pd.DataFrame(linearpipe.predict(test_x))
#df.columns = ["lin_x", "lin_y"]
#df.head()
#df_test.head()
#df_res[f"lr_{known}_{noise}"] = np.sqrt((test_y["output_x"] - df["lin_x"])**2 + (test_y["output_y"] - df["lin_y"])**2)

#print(df_res[f"lr_{known}_{noise}"].describe())

pickle.dump(linearpipe, open(f"../workdir/AIS-TrainedModel/lin_{known}_{noise}.csv", 'wb'))

In [ ]:
#with open(f"../workdir/AIS-TrainedModel/LinReg_{known}_{noise}.csv", 'rb') as f:
model = pickle.load(open(f"../workdir/AIS-TrainedModel/LinReg_{known}_{noise}.csv", 'rb'))

#model.predict(test_x)

df = pd.DataFrame(model.predict(test_x))
df.columns = ["linC_x", "linC_y"]
df.head()
#df_test.head()
df_res[f"lrC_{known}_{noise}"] = np.sqrt((test_y["output_x"] - df["linC_x"])**2 + (test_y["output_y"] - df["linC_y"])**2)

print(df_res[f"lrC_{known}_{noise}"].describe())

count    3.205680e+05
mean     9.308489e+00
std      1.290228e+01
min      3.935662e-07
25%      1.560694e+00
50%      4.313301e+00
75%      1.189245e+01
max      1.474059e+02
Name: lrC_3_0, dtype: float64


## Neural Network

https://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPRegressor.html

In [9]:
from sklearn.neural_network import MLPRegressor

mlppipe = Pipeline([("scaler", StandardScaler()), ("regressor", MLPRegressor((50)))])
mlppipe.fit(train_x, train_y)

#np.sqrt(test_y, mlppipe.predict(test_x))

#df_test = test.copy()
#df = pd.DataFrame(mlppipe.predict(test_x))
#df.columns = ["mlp_x", "mlp_y"]
#df.head()
#df_test.head()
#df_res[f"nn_{known}_{noise}"] = np.sqrt((test_y["output_x"] - df["mlp_x"])**2 + (test_y["output_y"] - df["mlp_y"])**2)

#print(df_res[f"nn_{known}_{noise}"].describe())

pickle.dump(mlppipe, open(f"../workdir/AIS-TrainedModel/mlp_{known}_{noise}.csv", 'wb'))

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## AutoSklearn   


In [13]:
from autosklearn.regression import AutoSklearnRegressor
import autosklearn.metrics
automl = AutoSklearnRegressor(
    time_left_for_this_task=120, memory_limit=100*1024, n_jobs=-1, metric=autosklearn.metrics.mean_squared_error
    )
automl.fit(train_x, train_y, dataset_name=f'known{known}')

ensemble_dict = automl.show_models()
print(ensemble_dict)




[WARNING] [2024-12-03 19:02:55,019:Client-AutoML(1):known3] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.045769)
[WARNING] [2024-12-03 19:02:55,019:Client-AutoML(1):known3] Capping the per_run_time_limit to 59.0 to have time for a least 2 models in each process.
[WARNING] [2024-12-03 19:02:55,189:Client-AutoMLSMBO(1)::known3] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2024-12-03 19:02:57,496:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:03:00,931:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:03:04,062:Client-EnsembleBuilder] No runs were available to build an ensemble from
{17: {'model_id': 17, 'rank': 1, 'cost': 26.247851085609945, 'ense

In [24]:
from autosklearn.regression import AutoSklearnRegressor
import autosklearn.metrics



automl = AutoSklearnRegressor(
    time_left_for_this_task=30, memory_limit=100*1024, n_jobs=-1, metric=autosklearn.metrics.mean_squared_error,
    #hidden_layer_sizes=(10, 20, 30),  # 3 hidden layers with 10, 20, 30 nodes each
    #, hidden_layer_depth(1), num_nodes_per_layer(3)
    )
automl.fit(train_x, train_y, dataset_name=f'known{known}')

ensemble_dict = automl.show_models()
print(ensemble_dict)


[WARNING] [2024-12-03 18:56:46,815:Client-AutoML(1):known3] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.006394)
[WARNING] [2024-12-03 18:56:46,815:Client-AutoML(1):known3] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2024-12-03 18:56:46,937:Client-AutoMLSMBO(1)::known3] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2024-12-03 18:56:49,986:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 18:56:52,723:Client-EnsembleBuilder] No runs were available to build an ensemble from
{21: {'model_id': 21, 'rank': 1, 'cost': 286.34164789763815, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorC

In [14]:
import numpy as np



#np.sqrt(mean_squared_error(test_y, automl.predict(test_x)))

#df_test = test.copy()
df = pd.DataFrame(automl.predict(test_x))
df.columns = ["autosk_x", "autosk_y"]
df.head()
#df_test.head()
df_res[f"as_{known}_{noise}"] = np.sqrt((test_y["output_x"] - df["autosk_x"])**2 + (test_y["output_y"] - df["autosk_y"])**2)

print(df_res[f"as_{known}_{noise}"].describe())

pickle.dump(automl, open(f"../workdir/AIS-TrainedModel/autosk_{known}_{noise}.csv", 'wb'))

count    320568.000000
mean          6.942214
std           8.795190
min           0.000854
25%           1.888821
50%           4.286965
75%           8.689399
max         203.463877
Name: as_3_0, dtype: float64


In [15]:
df_res.head()
#df_res.to_csv(f"../workdir/AIS-ModelFrames/aisdk-2023-11-08-xs_2_kkn_res_{known}_{noise}.csv", index=False)

,output_x,output_y,bl_3_0,lr_3_0,lrC_3_0,nn_3_0,as_3_0
0,3,0,21.758906,0.731286,0.731286,1.462513,3.771935
1,3,0,27.067508,13.418518,13.418518,14.435750,11.726620
2,3,0,20.390194,0.756131,0.756131,0.571709,3.810228
3,3,0,18.411410,1.013412,1.013412,0.948399,1.082789
4,3,0,24.396106,8.294446,8.294446,2.868002,52.813801


## Create Modells

In [ ]:
import pandas as pd
import sklearn as sk
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from sklearn.neural_network import MLPRegressor

from autosklearn.regression import AutoSklearnRegressor
import autosklearn.metrics

noise = 0
folder = "../workdir/AIS-ModelFrames/"
filename = f"aisdk-2023-11-08-xs_2_kkn"

for known in range(3, 13, 3):

    train = pd.read_csv(f"{folder}{filename}_train_{known}_0.csv")
    test = pd.read_csv(f"{folder}{filename}_test_{known}_0.csv")

    train_y = train.loc[:,["output_x","output_y"]]
    test_y = test.loc[:,["output_x","output_y"]]
    train_x = train.drop(columns=["output_x","output_y"])
    test_x = test.drop(columns=["output_x","output_y"])

    df_res = test_y.copy()

    df_res.rename(columns={"output_x": f"known_{known}_0", "output_y": f"noise_{known}_0"}, inplace=True)

    df_res[f"known_{known}_0"] = known
    df_res[f"noise_{known}_0"] = noise

    # linear regression

    linearpipe = Pipeline([("scaler", StandardScaler()), ("regressor", LinearRegression())])
    linearpipe.fit(train_x, train_y)
    pickle.dump(linearpipe, open(f"../workdir/AIS-TrainedModel/lin_{known}_{noise}.csv", 'wb'))


    # neural network

    
    mlppipe = Pipeline([("scaler", StandardScaler()), ("regressor", MLPRegressor((50)))])
    mlppipe.fit(train_x, train_y)
    pickle.dump(mlppipe, open(f"../workdir/AIS-TrainedModel/mlp_{known}_{noise}.csv", 'wb'))

    # autosklearn

    
    automl = AutoSklearnRegressor(
        time_left_for_this_task=120, memory_limit=100*1024, n_jobs=-1, metric=autosklearn.metrics.mean_squared_error
        )
    automl.fit(train_x, train_y, dataset_name=f'known{known}')

    ensemble_dict = automl.show_models()
    print(ensemble_dict)
    pickle.dump(automl, open(f"../workdir/AIS-TrainedModel/autosk_{known}_{noise}.csv", 'wb'))






/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[WARNING] [2024-12-03 19:35:35,137:Client-AutoML(1):known3] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.200925)
[WARNING] [2024-12-03 19:35:35,137:Client-AutoML(1):known3] Capping the per_run_time_limit to 59.0 to have time for a least 2 models in each process.
[WARNING] [2024-12-03 19:35:35,264:Client-AutoMLSMBO(1)::known3] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2024-12-03 19:35:37,308:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:35:40,512:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:35:40,970:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:35:43,649:Client-EnsembleBuilder] No runs

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[WARNING] [2024-12-03 19:41:57,293:Client-AutoML(1):known6] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (119.070501)
[WARNING] [2024-12-03 19:41:57,293:Client-AutoML(1):known6] Capping the per_run_time_limit to 59.0 to have time for a least 2 models in each process.
[WARNING] [2024-12-03 19:41:57,436:Client-AutoMLSMBO(1)::known6] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2024-12-03 19:42:00,801:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:42:05,104:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:42:08,516:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:42:12,183:Client-EnsembleBuilder] No runs

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[WARNING] [2024-12-03 19:54:29,364:Client-AutoML(1):known9] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (118.910699)
[WARNING] [2024-12-03 19:54:29,364:Client-AutoML(1):known9] Capping the per_run_time_limit to 59.0 to have time for a least 2 models in each process.
[WARNING] [2024-12-03 19:54:29,530:Client-AutoMLSMBO(1)::known9] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2024-12-03 19:54:33,406:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:54:39,072:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:54:44,337:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 19:54:50,333:Client-EnsembleBuilder] No runs

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[WARNING] [2024-12-03 20:07:54,102:Client-AutoML(1):known12] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (118.811019)
[WARNING] [2024-12-03 20:07:54,102:Client-AutoML(1):known12] Capping the per_run_time_limit to 59.0 to have time for a least 2 models in each process.
[WARNING] [2024-12-03 20:07:54,301:Client-AutoMLSMBO(1)::known12] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2024-12-03 20:07:59,409:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 20:08:04,811:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 20:08:13,346:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-12-03 20:08:18,665:Client-EnsembleBuilder] No r

## Check Modells under noise

In [ ]:
folder = "../workdir/AIS-ModelFrames/"
filename = f"aisdk-2023-11-08-xs_2_kkn"

for known in range(3, 13, 3):
    for noise in range(0, 11, 5):
        test = pd.read_csv(f"{folder}{filename}_test_{known}_{noise}.csv")

        test_y = test.loc[:,["output_x","output_y"]]
        test_x = test.drop(columns=["output_x","output_y"])

        if known == 3 and noise == 0:  
            df_res = test_y.copy()

            df_res.rename(columns={"output_x": f"known_{known}_0", "output_y": f"noise_{known}_0"}, inplace=True)

            df_res[f"known_{known}_0"] = known
            df_res[f"noise_{known}_0"] = noise
        else:
            df_res[f"known_{known}_{noise}"] = known
            df_res[f"noise_{known}_{noise}"] = noise

        # Baseline

        dummy_prediction = test_x.loc[:,[f"input_{known-1}_x", f"input_{known-1}_y"]]
        df_res[f"bl_{known}_{noise}"] = np.sqrt((test_y["output_x"] - dummy_prediction[f"input_{known-1}_x"])**2 + (test_y["output_y"] - dummy_prediction[f"input_{known-1}_y"])**2)
        #print(df_res[f"bl_{known}_{noise}"].describe())

        # Linear Regression


        linReg = pickle.load(open(f"../workdir/AIS-TrainedModel/lin_{known}_0.csv", 'rb'))
        df = pd.DataFrame(linReg.predict(test_x))
        df.columns = [f"lin_x_{known}_{noise}", f"lin_y_{known}_{noise}"]
        df_res[f"lr_{known}_{noise}"] = np.sqrt((test_y["output_x"] - df[f"lin_x_{known}_{noise}"])**2 + (test_y["output_y"] - df[f"lin_y_{known}_{noise}"])**2)

        #print(df_res[f"lr_{known}_{noise}"].describe())

        # Neural Network

        mlppipe = pickle.load(open(f"../workdir/AIS-TrainedModel/mlp_{known}_0.csv", 'rb'))
        df = pd.DataFrame(mlppipe.predict(test_x))
        df.columns = [f"mlp_x_{known}_{noise}", f"mlp_y_{known}_{noise}"]
        df_res[f"nn_{known}_{noise}"] = np.sqrt((test_y["output_x"] - df[f"mlp_x_{known}_{noise}"])**2 + (test_y["output_y"] - df[f"mlp_y_{known}_{noise}"])**2)
        #print(df_res[f"nn_{known}_{noise}"].describe())

        # AutoSklearn
        
        automl = pickle.load(open(f"../workdir/AIS-TrainedModel/autosk_{known}_0.csv", 'rb'))
        df = pd.DataFrame(automl.predict(test_x))
        df.columns = [f"autosk_x_{known}_{noise}", f"autosk_y_{known}_{noise}"]
        df_res[f"as_{known}_{noise}"] = np.sqrt((test_y["output_x"] - df[f"autosk_x_{known}_{noise}"])**2 + (test_y["output_y"] - df[f"autosk_y_{known}_{noise}"])**2)
        #print(df_res[f"as_{known}_{noise}"].describe())

        
        
df_res.to_csv(f"../workdir/AIS-TrainedModel/{filename}_result.csv", index=False)
    

In [5]:
df_res.head()

,known_3_0,noise_3_0,bl_3_0,lr_3_0,nn_3_0,as_3_0,known_3_5,noise_3_5,bl_3_5,lr_3_5,...,bl_12_5,lr_12_5,nn_12_5,as_12_5,known_12_10,noise_12_10,bl_12_10,lr_12_10,nn_12_10,as_12_10
0,3,0,21.758906,0.731286,1.676282,3.771935,3,5,15.854942,12.451557,...,20.157151,5.676367,4.694914,255.699599,12,10,13.887946,11.281088,8.814110,258.174573
1,3,0,27.067508,13.418518,16.801331,11.726620,3,5,26.729935,14.115557,...,22.726005,8.609695,9.824121,250.942444,12,10,37.864443,21.143863,14.349431,241.612243
2,3,0,20.390194,0.756131,0.063243,3.810228,3,5,24.724901,9.682861,...,30.291721,15.669231,16.235433,261.750880,12,10,28.979272,7.343083,10.707935,261.876005
3,3,0,18.411410,1.013412,0.334302,1.082789,3,5,17.751914,6.737658,...,32.924392,16.158755,15.962080,295.122930,12,10,51.155298,40.990162,40.832444,303.870463
4,3,0,24.396106,8.294446,2.363900,52.813801,3,5,27.183212,17.508919,...,15.706116,13.409705,3.712388,328.482145,12,10,34.300131,20.741553,22.409163,338.786213
